In [ ]:
!pip install kagglehub[hf-datasets]
!pip install azure-storage-blob
!pip install azure-storage-blob tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which i

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "data.json"

# Load the latest version
hf_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.HUGGING_FACE,
  "johntukey/github-dataset",
  file_path,
  # Provide any additional arguments like
  # sql_query, hf_kwargs, or pandas_kwargs. See
  # the documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterhugging_face
)

print("Hugging Face Dataset:", hf_dataset)

<ipython-input-5-55de83c65e75>:10: DeprecationWarning: load_dataset is deprecated and will be removed in future version.
  hf_dataset = kagglehub.load_dataset(


 30%|███       | 15.0G/50.0G [02:47<05:04, 123MB/s]

In [ ]:
!find / -type f -size +1000M -exec ls -lh {} \; 2>/dev/null

-rw-r--r-- 1 root root 1.1G Apr 23 13:47 /usr/local/lib/python3.11/dist-packages/tensorflow/libtensorflow_cc.so.2
-rwxr-xr-x 1 root root 1.3G Apr 23 13:41 /usr/local/lib/python3.11/dist-packages/libcuvs/lib64/libcuvs.so
-rwxr-xr-x 1 root root 1.9G Apr 23 13:41 /usr/local/lib/python3.11/dist-packages/libcugraph/lib64/libcugraph.so
-rw-r--r-- 1 root root 51G Apr 25 07:04 /root/.cache/kagglehub/datasets/johntukey/github-dataset/versions/1/data.json


In [ ]:
!pip install azure-storage-blob tqdm

import os
from azure.storage.blob import BlobServiceClient
from tqdm import tqdm
from google.colab import userdata


class Progress:
    def __init__(self, file_path):
        self.total_size = os.path.getsize(file_path)
        self.pbar = tqdm(total=self.total_size, unit='B', unit_scale=True, desc="Uploading")
        self.uploaded = 0

    def __call__(self, response):
        # Get number of bytes uploaded from the response
        current = response.context['upload_stream_current']
        delta = current - self.uploaded
        self.uploaded = current
        self.pbar.update(delta)

# Azure details
account_name = userdata.get('AZURE_ACCOUNT_NAME')
account_key = userdata.get('AZURE_STORAGE_KEY')
container_name = "kaggle-datasets"
file_path = "/root/.cache/kagglehub/datasets/johntukey/github-dataset/versions/1/data.json"
blob_name = "github-dataset-full.json"

# Set up Azure connection
conn_str = f"DefaultEndpointsProtocol=https;AccountName={account_name};AccountKey={account_key};EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

# Upload file with progress
progress = Progress(file_path)
with open(file_path, "rb") as data:
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    blob_client.upload_blob(data, overwrite=True, raw_response_hook=progress)

print(f"Upload completed for '{blob_name}'")

Uploading: 100%|██████████| 53.7G/53.7G [1:02:55<00:00, 16.0MB/s]

TypeError: unsupported operand type(s) for -: 'NoneType' and 'int'